In [1]:
CREATE TABLE dbo.[User]
(
    UserID INT IDENTITY(1,1) NOT NULL,
    LoginName NVARCHAR(40) NOT NULL,
    PasswordHash BINARY(64) NOT NULL,
    FirstName NVARCHAR(40) NULL,
    LastName NVARCHAR(40) NULL,
    Age INT NULL,
    CONSTRAINT [PK_User_UserID] PRIMARY KEY CLUSTERED (UserID ASC)
)

Commands completed successfully.

Total execution time: 00:00:00.077

In [6]:


ALTER PROCEDURE dbo.AddUser
    @pLogin NVARCHAR(50), 
    @pPassword NVARCHAR(50),
    @pFirstName NVARCHAR(40) = NULL, 
    @pLastName NVARCHAR(40) = NULL,
    @pAge INT = NULL,
    @responseMessage NVARCHAR(250) OUTPUT
AS
BEGIN
    SET NOCOUNT ON

    DECLARE @salt UNIQUEIDENTIFIER=NEWID()
    DECLARE @hash BINARY(64) = HASHBYTES('SHA2_512', @pPassword+CAST(@salt AS NVARCHAR(36)))
    BEGIN TRY

        INSERT INTO dbo.[User] (LoginName, PasswordHash, Salt, FirstName, LastName, Age)
        VALUES(@pLogin, @hash, @salt, @pFirstName, @pLastName, @pAge)

       SET @responseMessage='Success'

    END TRY
    BEGIN CATCH
        SET @responseMessage=ERROR_MESSAGE() 
    END CATCH

END

Commands completed successfully.

Total execution time: 00:00:00.057

In [7]:
-- Inserting Data

TRUNCATE TABLE [dbo].[User]

DECLARE @responseMessage NVARCHAR(250)

EXEC dbo.AddUser
          @pLogin = N'Admin',
          @pPassword = N'123',
          @pFirstName = N'Admin',
          @pLastName = N'Administrator',
          @responseMessage=@responseMessage OUTPUT

SELECT UserID, LoginName, PasswordHash, Salt, FirstName, LastName, Age
FROM [dbo].[User]

(1 row affected)

Total execution time: 00:00:00.072

UserID,LoginName,PasswordHash,Salt,FirstName,LastName,Age
1,Admin,0x427C33D3FAF4615579EB339D8E4361B83AE493FC6E0C66E21D4427D6293A9A39D6939CE4E7914B3D30DFD01E852BC14BB0A10D5E9E3E3CE65726444FBB7E34B5,9df17903-81f4-4bce-ba16-268cc60e8f74,Admin,Administrator,NULL


In [8]:
-- Login

CREATE PROCEDURE dbo.Login
    @pLoginName NVARCHAR(254),
    @pPassword NVARCHAR(50),
    @responseMessage NVARCHAR(250)='' OUTPUT
AS
BEGIN

    SET NOCOUNT ON

    DECLARE @userID INT

    IF EXISTS (SELECT TOP 1 UserID FROM [dbo].[User] WHERE LoginName=@pLoginName)
    BEGIN
        SET @userID=(SELECT UserID FROM [dbo].[User] WHERE LoginName=@pLoginName AND PasswordHash=HASHBYTES('SHA2_512', @pPassword+CAST(Salt AS NVARCHAR(36))))

       IF(@userID IS NULL)
           SET @responseMessage='Incorrect password'
       ELSE 
           SET @responseMessage='User successfully logged in'
    END
    ELSE
       SET @responseMessage='Invalid login'

END

Commands completed successfully.

Total execution time: 00:00:00.074

In [10]:
-- Checking the Login Function

DECLARE	@responseMessage nvarchar(250)

--Correct login and password
EXEC	dbo.Login
		@pLoginName = N'Admin',
		@pPassword = N'123',
		@responseMessage = @responseMessage OUTPUT

SELECT	@responseMessage as N'@responseMessage'

--Incorrect login
EXEC	dbo.Login
		@pLoginName = N'Admin1', 
		@pPassword = N'123',
		@responseMessage = @responseMessage OUTPUT

SELECT	@responseMessage as N'@responseMessage'

--Incorrect password
EXEC	dbo.Login
		@pLoginName = N'Admin', 
		@pPassword = N'1234',
		@responseMessage = @responseMessage OUTPUT

SELECT	@responseMessage as N'@responseMessage'

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.072

@responseMessage
User successfully logged in


@responseMessage
Invalid login


@responseMessage
Incorrect password
